# Visualization Function Testing Notebook

This notebook tests the visualization functions `visualize_sample_slice` with configurable dataset and domain settings. It focuses on ground truth visualization and skips 3D visualization as requested.

## Features:
- Tests CHAOS and MMWHS datasets
- Supports CT and MR domains  
- Configurable visualization parameters
- Ground truth visualization testing
- Error handling and cleanup

In [1]:
import torch
import numpy as np
from pathlib import Path

from src.datasets.registry import get_dataset

## Configuration

Change these constants to test different datasets and domains:
- **DATASET_NAME**: "CHAOS" or "MMWHS"
- **DOMAIN**: "CT" or "MR" 
- **ENCODER_TYPE**: "resnet" or "swin_unetr"
- **USE_SEMANTIC_HEAD**: Enable/disable semantic guidance

In [2]:
# Configuration constants
DATASET_NAME = "MMWHS"  # Change to "MMWHS" if needed
DOMAIN = "CT"  # Change to "CT" if needed
ENCODER_TYPE = "swin_unetr"  # Change to "swin_unetr" if needed
BATCH_SIZE = 1
NUM_WORKERS = 1

In [3]:
def preprocess(x):
    # Only scale down to (96, 128, 128) if larger
    target_shape = (96, 128, 128)
    img = x["image"].float()
    lbl = x["label"].float() if x["label"] is not None else None

    if img.shape[-3:] != target_shape:
        img = torch.nn.functional.interpolate(
            img.unsqueeze(0),
            size=target_shape,
            mode="trilinear",
            align_corners=False,
        ).squeeze(0)
        if lbl is not None:
            lbl = (
                torch.nn.functional.interpolate(
                    lbl.unsqueeze(0), size=target_shape, mode="nearest"
                )
                .squeeze(0)
                .long()
            )
    else:
        if lbl is not None:
            lbl = lbl.long()

    return {"image": img, "label": lbl}

## Ground Truth Visualization Testing

This section loads the dataset and tests the `visualize_sample_slice` function with ground truth data.

In [ ]:
def test_ground_truth_visualization():
    """Test visualization functions with ground truth data."""
    print("🔍 Testing Ground Truth Visualization...")
    print(f"Dataset: {DATASET_NAME}, Domain: {DOMAIN}")

    # Load dataset
    dataset = get_dataset(
        dataset_name=DATASET_NAME,
        domain=DOMAIN,
        base_path=Path("data"),
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        preprocess=preprocess,
        slice_2d=False,  # Use 3D data
    )

    # Get a sample from the dataset
    loader = dataset.train_loader
    batch = next(iter(loader))

    # Extract a single sample
    sample = {
        "image": batch["image"],  # Keep batch dimension for inference
        "label": batch["label"],  # Keep batch dimension for inference
    }
    return dataset, sample, batch


def test_inference_and_visualization(dataset, batch):
    """Test inference with semantic head training and visualize both GT and prediction."""
    print("🚀 Testing Inference with Semantic Head Training...")
    print(f"Dataset: {DATASET_NAME}, Domain: {DOMAIN}")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # Get model with semantic head
    model = dataset.get_model(ENCODER_TYPE)

    # Restore the checkpoint
    checkpoint_path = Path("checkpoints", f"{DATASET_NAME}_{DOMAIN}_3d_finetuned.pth")
    finetuned_state_dict = torch.load(checkpoint_path, map_location=device).state_dict()
    model.encoder.load_state_dict(finetuned_state_dict)

    model.to(device)
    batch["image"] = batch["image"].to(device)
    batch["label"] = batch["label"].to(device)

    # Run inference
    print("🔮 Running inference...")
    outputs = model(batch["image"])
    preds = torch.argmax(outputs, dim=1, keepdim=True)

    print(f"Prediction unique values: {torch.unique(preds)}")

    return preds

## Dataset Visualization Method Test

Test the `dataset.visualize_sample_slice()` method with the loaded data. This section will show both ground truth and prediction visualizations for the **same sample** to enable direct comparison. The dataset-specific implementation automatically applies the correct rotation and flip parameters for each dataset type.

In [5]:
## Dataset Visualization Method Test
dataset, sample, batch = test_ground_truth_visualization()

# Test dataset.visualize_sample_slice method
print("\n📊 Testing dataset.visualize_sample_slice method...")
# Squeeze batch and channel dimensions for visualization
sample["image"] = sample["image"].squeeze().permute(2, 1, 0).cpu().numpy()
sample["label"] = sample["label"].squeeze().permute(2, 1, 0).cpu().numpy()
# Use the dataset method instead of standalone function
# dataset.visualize_sample_slice(sample)

🔍 Testing Ground Truth Visualization...
Dataset: MMWHS, Domain: CT
Sample image shape: torch.Size([1, 1, 96, 128, 128])
Sample label shape: torch.Size([1, 1, 96, 128, 128])

📊 Testing dataset.visualize_sample_slice method...


In [7]:
## Prediction Visualization

# Import required modules for safe loading
from monai.networks.nets import SwinUNETR
from monai.networks.nets.swin_unetr import SwinTransformer
from monai.networks.blocks.patchembedding import PatchEmbed
from torch.nn.modules.conv import Conv3d
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.container import ModuleList
from monai.networks.nets.swin_unetr import BasicLayer
from monai.networks.nets.swin_unetr import SwinTransformerBlock
from torch.nn.modules.normalization import LayerNorm
from monai.networks.nets.swin_unetr import WindowAttention
from torch.nn.modules.linear import Linear
from torch.nn.modules.activation import Softmax
from torch.nn.modules.linear import Identity
from monai.networks.blocks.mlp import MLPBlock
from torch.nn.modules.activation import GELU
from monai.networks.nets.swin_unetr import PatchMerging
from monai.networks.blocks.unetr_block import UnetrBasicBlock
from monai.networks.blocks.dynunet_block import UnetResBlock
from monai.networks.blocks.convolutions import Convolution
from torch.nn.modules.activation import LeakyReLU
from torch.nn.modules.instancenorm import InstanceNorm3d
from monai.networks.blocks.unetr_block import UnetrUpBlock
from torch.nn.modules.conv import ConvTranspose3d
from monai.networks.blocks.dynunet_block import UnetOutBlock

# Run inference on the SAME sample
print("\n🔮 Testing dataset.visualize_sample_slice method with PREDICTIONS...")
with torch.serialization.safe_globals(
    [
        SwinUNETR,
        SwinTransformer,
        PatchEmbed,
        Conv3d,
        Dropout,
        ModuleList,
        BasicLayer,
        SwinTransformerBlock,
        LayerNorm,
        WindowAttention,
        Linear,
        Softmax,
        Identity,
        MLPBlock,
        GELU,
        PatchMerging,
        UnetrBasicBlock,
        UnetResBlock,
        Convolution,
        LeakyReLU,
        InstanceNorm3d,
        UnetrUpBlock,
        ConvTranspose3d,
        UnetOutBlock,
    ]
):
    # Get predictions using the same batch
    preds = test_inference_and_visualization(dataset, batch)

# Prepare prediction sample for visualization
pred_sample = {"image": sample["image"], "label": None}  # Will be filled below

# Apply dataset-specific label mapping for predictions
preds = dataset.encode(preds)
print("Prediction unique values:", np.unique(pred_sample["label"]))
# Use the dataset method for prediction visualization
dataset.visualize_sample_slice(sample)
dataset.visualize_sample_slice(pred_sample)


🔮 Testing dataset.visualize_sample_slice method with PREDICTIONS...
🚀 Testing Inference with Semantic Head Training...
Dataset: MMWHS, Domain: CT
Using device: cpu


AttributeError: 'MMWHS' object has no attribute 'num_classes'